In [4]:
import os
import pandas as pd
log_path = '/Users/subashkhanal/Desktop/paper1_results/GeneExpr'
feats_path = './results_final/'

In [2]:
os.listdir(log_path)

['Features_ranked_for_CN_AD_xgb_25_no_prune_fromModel_correct.csv',
 'Features_ranked_for_CN_EMCI_xgb_50_no_prune_fromModel_correct.csv',
 'top_genes_exper.ipynb',
 'results_final',
 'GeneExpr_sweep_without_extra.log',
 'ttest',
 'Features_ranked_for_LMCI_AD_xgb_25_no_prune_fromModel_correct.csv',
 '.ipynb_checkpoints',
 'results_without_Extra.csv',
 'Features_ranked_for_EMCI_AD_xgb_25_no_prune_fromModel_correct.csv']

In [23]:
with open('GeneExpr_sweep_without_extra.log','r') as infile:
    text = infile.read().strip().split('For parameters: ')[1:]

In [40]:
text[0]

"('CN_AD', 'xgb', 'correct', 25, 'not_extra', 'prune', 'fromModel')\nBest hyperparameters for xgb : (10, 2)\nbest Macro ACC: 0.5667735042735043 best Macro AUC: 0.6190972222222222\nAvg number of features AFTER FEATURE SELECTION: 10\nfor Avg 10 Features\nMean Balanced Accuracy: 0.5667735042735043\nMean AUC: 0.6190972222222222\nEND OF THE EXPERIMENT\n0.5667735042735043 0.6190972222222222\n\n\n"

In [49]:
results = []
for t in text:
    group = t.split('\n')[0].split(',')[0].replace('(','')
    N = int(t.split('\n')[0].split(',')[3].strip())
    prune = t.split('\n')[0].split(',')[5]
    F = int(t.split('\n')[3].split(':')[1].strip())
    params = t.split('\n')[1].split(':')[1].strip()
    ACC = float(t.split('END OF THE EXPERIMENT')[1].strip().split(' ')[0])
    AUC = float(t.split('END OF THE EXPERIMENT')[1].strip().split(' ')[1])
    result = [group,N, prune, F, params, ACC, AUC]
    results.append(result)
df = pd.DataFrame(results,columns=['group','N','prune','F','params','ACC','AUC'])

In [51]:
df.to_csv('results_without_Extra.csv')

In [5]:
os.listdir(feats_path)

['LMCI_AD_xgb_25_no_prune_fromModel_correct_whole_imp.csv',
 'Features_ranked_for_CN_AD_xgb_25_no_prune_fromModel_correct.csv',
 'ROC_for:CN_EMCI_xgb_50_no_prune_fromModel_correct.png',
 'Features_ranked_for_CN_EMCI_xgb_50_no_prune_fromModel_correct.csv',
 'ROC_for:CN_AD_xgb_25_no_prune_fromModel_correct.png',
 'CN_AD_xgb_25_no_prune_fromModel_correct_whole_imp.csv',
 'ROC_for:LMCI_AD_xgb_25_no_prune_fromModel_correct.png',
 'Features_ranked_for_LMCI_AD_xgb_25_no_prune_fromModel_correct.csv',
 'CN_EMCI_xgb_50_no_prune_fromModel_correct_whole_imp.csv',
 'ROC_for:EMCI_AD_xgb_25_no_prune_fromModel_correct.png',
 'Features_ranked_for_EMCI_AD_xgb_25_no_prune_fromModel_correct.csv',
 'EMCI_AD_xgb_25_no_prune_fromModel_correct_whole_imp.csv']

In [97]:
def clean_feats(feats): 
    final_feats = []
    for f in feats:
        if len(f.split('_at_')) > 1:
            final_feats.append(f.split('_at_')[1])
        else:
            final_feats.append(f)
    return final_feats

In [6]:
best_map = {
    'CN_AD':'CN_AD_xgb_25_no_prune_fromModel_correct_whole_imp.csv',
    'CN_EMCI':'CN_EMCI_xgb_50_no_prune_fromModel_correct_whole_imp.csv',
    'EMCI_AD':'EMCI_AD_xgb_25_no_prune_fromModel_correct_whole_imp.csv',
    'LMCI_AD':'LMCI_AD_xgb_25_no_prune_fromModel_correct_whole_imp.csv'
    }

In [7]:
groups = ['CN_AD', 'CN_EMCI', 'EMCI_AD', 'LMCI_AD']
final_dict= {}

In [14]:
filepath = '/Users/subashkhanal/Desktop/paper1_results/GeneExpr/results_final'

In [15]:
df = pd.read_csv(os.path.join(filepath,best_map[groups[0]]))
df.head()

,Unnamed: 0,features_fold0,importance_fold0,features_fold1,importance_fold1,features_fold2,importance_fold2,features_fold3,importance_fold3,features_fold4,importance_fold4
0,0,AGE,0.051372,AGE,0.052416,AGE,0.064489,AGE,0.035081,AGE,0.048787
1,1,PTEDUCAT,0.064223,PTEDUCAT,0.048959,PTEDUCAT,0.026712,PTEDUCAT,0.022104,PTEDUCAT,0.059664
2,2,11761978_at_ARHGEF12,0.096794,11719477_a_at_FASTKD5,0.078151,11763538_x_at_PRR5,0.103760,11720732_a_at_SUMF1,0.138276,11738864_a_at_ABHD12B,0.000000
3,3,11750580_x_at_ARHGEF12,0.000000,11755705_a_at_SMIM5,0.072915,AFFX-r2-Bs-phe-M_at_nan,0.156322,11747479_x_at_MLST8,0.000000,11743803_s_at_SEPHS2,0.038672
4,4,11758611_s_at_TRIM10,0.026200,11752515_a_at_ODC1,0.000000,11726810_s_at_C19ORF77,0.069105,11754381_a_at_TMOD1,0.000000,11750558_s_at_ZNF271,0.053384


In [105]:
FOLDS = 5
def importance_avg(group,threshold=0.01):
    df = pd.read_csv(os.path.join(filepath,best_map[group]))
    all_feats = []
    for fold in range(FOLDS):
        all_feats = all_feats + list(df['features_fold'+str(fold)])
    all_feats = list(set(all_feats)) # Overall features in all folds
    df_avg = pd.DataFrame(columns=['features','importance'])
    df_avg['features'] = all_feats
    df_avg['importance'] = np.zeros(len(all_feats,))
    for i in range(len(df_avg)):
        tmp_imp = np.zeros(FOLDS,)
        current_feat = df_avg.iloc[i]['features']
        for fold in range(FOLDS):
            temp_df =df[['features_fold'+str(fold),'importance_fold'+str(fold)]]
            if current_feat in list(temp_df['features_fold'+str(fold)]):
                tmp_imp[fold] = temp_df[temp_df['features_fold'+str(fold)] == current_feat]['importance_fold'+str(fold)].item()
        df_avg.at[i,'features'] = current_feat
        df_avg.at[i,'importance'] = np.mean(tmp_imp)
    df_avg_sorted = df_avg.sort_values(by=['importance'],ascending=False)
    df_avg_sorted = df_avg_sorted[df_avg_sorted['importance']>=threshold]
    clean_features = clean_feats(df_avg_sorted['features'])
    df_avg_sorted['features'] = clean_features
    df_avg_sorted.groupby("features")["importance"].mean()
    df_avg_sorted = df_avg_sorted[df_avg_sorted.features != 'nan']
    return df_avg_sorted

In [117]:
df = importance_avg(groups[0],0.01)
df.to_csv(os.path.join(feats_path,'final_gene_ranked_for'+groups[0]+'.csv'))
df = importance_avg(groups[1],0.01)
df.to_csv(os.path.join(feats_path,'final_gene_ranked_for'+groups[1]+'.csv'))
df = importance_avg(groups[2],0.01)
df.to_csv(os.path.join(feats_path,'final_gene_ranked_for'+groups[2]+'.csv'))